In [1]:
import torch
from d2l import torch as d2l

There are many other sequence learning tasks contexts where it is perfectly fine to condition the prediction at every time step on <font color='red'>both the leftward and the rightward context</font>. Consider, for example, part of <font color='red'>speech detection</font>. Why shouldn't we take the context in both directions into account when assessing the part of speech associated with a given word?

In [2]:
class BiRNNScratch(d2l.Module):
    def __init__(self, num_inputs, num_hiddens, sigma=0.01):
        super().__init__()
        self.save_hyperparameters()
        self.f_rnn = d2l.RNNScratch(num_inputs, num_hiddens, sigma)
        self.b_rnn = d2l.RNNScratch(num_inputs, num_hiddens, sigma)
        self.num_hiddens *= 2  # The output dimension will be doubled

    def forward(self, inputs, Hs=None):
        f_H, b_H = Hs if Hs is not None else (None, None)
        f_outputs, f_H = self.f_rnn(inputs, f_H)
        b_outputs, b_H = self.b_rnn(reversed(inputs), b_H)
        outputs = [torch.cat((f, b), -1) for f, b in zip(f_outputs, reversed(b_outputs))]
        return outputs, (f_H, b_H)